In [ ]:
import weave
from weave.legacy.weave import weave_internal
from weave.legacy.weave.ecosystem.all import langchain
weave.use_frontend_devmode()

In [ ]:
weave.legacy.weave.panels.Board(
    vars={
        'gpt35': langchain.chat_openai("gpt-3.5-turbo", 0.7),
        'gpt4': langchain.chat_openai("gpt-4", 0.7),
        'chat_model': lambda gpt4, gpt35: gpt4,
        'director': 'Wes Anderson',
        'director_emotion': 'worldy',
        'film_style': 'rollerblading documentary',
        'number_of_panes': 4,
        'story_gen_prompt': """
            You are {director} creating the next {film_style} film. Today you're feeling {director_emotion}.
            In initial sketches, you write a {number_of_panes} step sketch of the plot of your film.
            You include specific details about how the plot proceeds.
            What do you write? Since you are so creative, you can overcome any limitations you might
            think you have when it comes to writing style. In this case, you write your outline as a json list
            with one item for each step.
            """,
        'prompt': lambda director, director_emotion, film_style, number_of_panes, story_gen_prompt: story_gen_prompt.format(weave.legacy.weave.ops.dict_(
            director= director,
            director_emotion=director_emotion,
            film_style=film_style,
            number_of_panes=number_of_panes)),
        'storyline': lambda prompt, chat_model: chat_model.predict(prompt),
        'storyline_parsed': lambda storyline: storyline.json_parse_list(),
        'panel_prompt': 'A storyboard pane for a movie by {director} for their next {film_style} film, The scene description is: {panel}',
        'storyboard_panes': lambda storyline_parsed, panel_prompt, director, film_style:
                storyline_parsed.map(lambda row:
                    panel_prompt.format(weave.legacy.weave.ops.dict_(
                        panel=row.json_dumps(),
                        director=director,
                        film_style=film_style,
                    ))).stable_diffusion()
    },
    panels=[      
        weave.legacy.weave.panels.BoardPanel(
            lambda storyline_parsed: storyline_parsed,
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=6)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda storyboard_panes: weave.legacy.weave.panels.Each(storyboard_panes),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=12)
        )
    ]
)

In [ ]:
# imperative version

gpt35 = langchain.chat_openai("gpt-3.5-turbo", 0.7)
gpt4 = langchain.chat_openai("gpt-4", 0.7)
chat_model = gpt4

director = 'Wes Anderson'
film_style = 'great american'
number_of_panes = 4
story_gen_prompt = weave.save("""
You are {director} creating the next {film_style} film.
In initial sketches, you write a {number_of_panes} step sketch of the plot of your film.
You include specific details about how the plot proceeds.
What do you write? Since you are so creative, you can overcome any limitations you might
think you have when it comes to writing style. In this case, you write your outline as a json list
with one item for each step.
""",
    name='story_gen_prompt')
storyline = gpt35.predict(story_gen_prompt.format({
    'director': director,
    'film_style': film_style,
    'number_of_panes': number_of_panes}))
storyline_parsed = storyline.json_parse()
#storyline_parsed

In [ ]:
#weave.legacy.weave.panels.Each(storyline_parsed.map(lambda row: row.json_dumps()).stable_diffusion())